In [1]:
import ollama
ollama.pull("llama3:8b")

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [2]:
# --- Classification Function ---
def classify_urdu_question(urdu_question):
    prompt = f"""
You are a reasoning expert. Your task is to classify the complexity of a question written in Urdu into one of two categories:

- "singlehop": A question that can be answered with one reasoning step or simple fact retrieval.
- "multihop": A question that requires multiple reasoning steps or combining information from different parts.

Here are a few examples:

Example 1:
Question (Urdu): پاکستان کا دارالحکومت کیا ہے؟
Classification: singlehop

Example 2:
Question (Urdu): وہ شخص کون تھا جو قائد اعظم کے بعد گورنر جنرل بنا؟
Classification: singlehop

Example 3:
Question (Urdu): وہ سائنسدان کون ہے جس نے نظریہ ارتقاء پیش کیا، اور اس کا اثر بیسویں صدی کی حیاتیات پر کیا پڑا؟
Classification: multihop

Now classify the following question:

Question (Urdu): {urdu_question}
Classification:"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[{"role": "user", "content": prompt}]
        )
        reply = response['message']['content'].strip().lower()

        # Normalize output
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {urdu_question}\n↪ {e}")
        return "error"


In [14]:
def decompose_urdu_query(urdu_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
**Role**: You are an expert Urdu linguistic analyst specializing in question decomposition. Your task is to break down complex Urdu questions into their fundamental components.

**Task Instructions**:
1. Carefully analyze the given Urdu question to identify its core components
2. Extract exactly 2 sub-questions that:
   - Are necessary to answer the main question
   - Cover distinct aspects of the problem
   - Have clear logical progression (answer to q1 helps answer q2)
3. Both sub-questions must:
   - Be in proper Urdu language
   - Be grammatically correct
   - Be clear and concise
   - Use relevant domain terminology

**Output Format Requirements**:
- Use EXACTLY this format:
  q1: [پہلا ذیلی سوال]
  q2: [دوسرا ذیلی سوال]
- Each sub-question must be on a new line
- Do not include any additional commentary or explanation
- Do not number the questions (use only q1:/q2: prefixes)

**Example 1**:
Input: اگر لاہور میں فضائی آلودگی کی سطح دہلی سے زیادہ ہے اور فضائی آلودگی پھیپھڑوں کے کینسر کا سبب بن سکتی ہے، تو لاہور کے رہائشیوں کو کس قسم کے طبی چیک اپ کروانے چاہئیں؟
Output:
q1: لاہور اور دہلی میں فضائی آلودگی کی سطح کا موازنہ کیا ہے؟
q2: فضائی آلودگی پھیپھڑوں کے کینسر کا سبب کیسے بنتی ہے؟

**Example 2**:
Input: اگر کراچی میں بجلی کے نرخ 30% بڑھ گئے ہیں اور یہ صنعتوں کو متاثر کر رہا ہے، تو حکومت کو کون سی سبسڈیاں دینی چاہئیں؟
Output:
q1: کراچی میں بجلی کے نرخوں میں اضافے کی موجودہ شرح کیا ہے؟
q2: بجلی کے مہنگے ہونے سے صنعتوں پر کس قسم کے اثرات مرتب ہو رہے ہیں؟

**Current Task**:
Input: {urdu_query}
Output:
"""
    
    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048
            }
        )
        
        output = response['response'].strip()
        
        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()
        
        return result if len(result) == 2 else {}
    
    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}


In [61]:
def query_context_relevance_check(query_urdu: str, context_urdu: str) -> bool:
    prompt = f"""
You are a binary classifier.

Your task is to decide whether the following Urdu *context* is relevant to the Urdu *question*. You must answer ONLY with **True** or **False** — no explanation, no commentary, just one word: True or False.

Criteria:
- If the context helps answer the question directly or indirectly, reply: True
- If the context is unrelated, confusing, or insufficient, reply: False

IMPORTANT:
- Do NOT explain your answer.
- Do NOT include any additional comments.
- Just respond with: True or False

---

Question (Urdu): {query_urdu}

Context (Urdu): {context_urdu}

Answer (True/False):
"""

    try:
        response = ollama.chat(
            model="llama3:8b",
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response['message']['content'].strip().lower()
        return answer == 'true'
    except Exception as e:
        print(f"Error during relevance check: {e}")
        return False


In [62]:
from sentence_transformers import SentenceTransformer
import pickle
import faiss
import os


def load_retriever(
    index_path: str,
    chunks_path: str,
    model_path: str = 'C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\embedding_model'
):
    # Load sentence transformer model
    if os.path.exists(model_path):
        model = SentenceTransformer(model_path)
    else:
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        model.save(model_path)

    # Load FAISS index
    index = faiss.read_index(index_path)

    # Load stored chunks
    with open(chunks_path, "rb") as f:
        chunks_list = pickle.load(f)

    return model, index, chunks_list



In [63]:
model, index, chunks_list = load_retriever(
        index_path="C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\vector_db\\paragraphs\\urdu_faiss_index_for_100_para.index",
        chunks_path="C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\data_storage\\parachunks\\urdu_chunks_for_100_para.pkl"
    )

In [75]:
def retrieve_documents(query, model, index, chunks_list, k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k)
    retrieved_chunks = [chunks_list[i] for i in I[0]]
    return retrieved_chunks

In [129]:
def get_context_of_multihop_without_parallel(query,model=model,index=index,chunks_list=chunks_list,k=3):


    classification = classify_urdu_question(query)


    if classification == "singlehop":
        retrieved_context = retrieve_documents(query, model, index, chunks_list, k)
        return retrieved_context
        

    if classification == "multihop":
        decomposition = decompose_urdu_query(query)
        q1 = decomposition.get("q1", "")
        q2 = decomposition.get("q2", "")

        main_context = retrieve_documents(q1, model, index, chunks_list, k)

        for i in range(min(len(main_context), k)):
            intermediate_ctx = main_context[i]
            
            combined_query = q1 + intermediate_ctx + q2
            
            second_hop_contexts = retrieve_documents(combined_query, model, index, chunks_list, k)
            
            for ctx in second_hop_contexts:
                if query_context_relevance_check(query, ctx):
                    main_context.append(ctx)
        
        return main_context    
 


In [130]:

import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def expand_multihop_context(intermediate_ctx, query, q1, q2, model, index, chunks_list, k):
    try:
        combined_query = q1 + intermediate_ctx + q2
        second_hop_contexts = retrieve_documents(combined_query, model, index, chunks_list, k)

        relevant_contexts = []

        with ThreadPoolExecutor() as inner_executor:
            futures = [
                inner_executor.submit(query_context_relevance_check, query, ctx)
                for ctx in second_hop_contexts
            ]

            for i, future in enumerate(as_completed(futures)):
                try:
                    if future.result():
                        relevant_contexts.append(second_hop_contexts[i])
                except Exception as e:
                    print("Error during relevance check:", e)

        return relevant_contexts

    except Exception as e:
        print("Error in expand_multihop_context:", e)
        return []


def get_context_of_multihop(query, type, model=model, index=index, chunks_list=chunks_list, k=3):
    # Measure classification time
    start_classification = time.time()
    classification = classify_urdu_question(query)
    classification_time = time.time() - start_classification

    if type == "easy":
        decomposition_time = 0.0
        start_retrieval = time.time()
        context = retrieve_documents(query, model, index, chunks_list, k)
        retrieval_time = time.time() - start_retrieval
        return context, type, classification_time, decomposition_time, retrieval_time

    else:
        start_decomposition = time.time()
        decomposition = decompose_urdu_query(query)
        q1 = decomposition.get("q1", "")
        q2 = decomposition.get("q2", "")
        decomposition_time = time.time() - start_decomposition

        start_retrieval = time.time()
        main_context = retrieve_documents(q1, model, index, chunks_list, k)
        additional_contexts = []

        with ThreadPoolExecutor() as executor:
            futures = [
                executor.submit(expand_multihop_context, ctx, query, q1, q2, model, index, chunks_list, k)
                for ctx in main_context[:k]
            ]

            for future in as_completed(futures):
                result = future.result()
                additional_contexts.extend(result)

        main_context.extend(additional_contexts)
        retrieval_time = time.time() - start_retrieval

        return main_context, classification, classification_time, decomposition_time, retrieval_time


In [132]:
ollama.pull('hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [137]:
import ollama

def generate_using_llama3(context, query):
    prompt = f"""You are a helpful assistant. You will be given a context and a question, both in Urdu.
Your task is to answer the question using the context only. 
Your answer should be **clear, concise, and entirely in Urdu**.
Context:
{context}

Question:
{query}

Answer in Urdu:"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print("Error during generation:", e)
        return "جواب پیدا کرنے میں خرابی ہوئی۔"

import ollama

def generate_using_alif(context, query, alif_model='hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16'):


    prompt = f"""آپ کو ایک سوال اور اس سے متعلق ایک سیاق و سباق دیا گیا ہے۔ براہ کرم سیاق و سباق کا بغور مطالعہ کریں اور اسی کی بنیاد پر درست، مختصر اور جامع جواب دیں۔

### سوال:
{query}

### سیاق و سباق:
{context}

### جواب:
"""

    response = ollama.chat(
        model=alif_model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=False
    )

    return response['message']['content']




In [138]:
import time

def multihop_handling_LQR(query, type, model=model, index=index, chunks_list=chunks_list, k=3):
    # Step 1: Get context and timings
    context, classification, classification_time, decomposition_time, retrieval_time = get_context_of_multihop(
        query, type, model=model, index=index, chunks_list=chunks_list, k=k
    )

    # Flatten context if it's a list of strings
    if isinstance(context, list):
        combined_context = "\n".join(context)
    else:
        combined_context = context

    # Step 2: Generate answer and measure time
    start_gen = time.time()
    final_answer = generate_using_alif(combined_context,query)
    generation_time = time.time() - start_gen

    # Step 3: Compute total time
    total_time = classification_time + decomposition_time + retrieval_time + generation_time

    return {
        "classification": classification,
        "retrieved_context": context,
        "final_answer": final_answer,
        "timings": {
            "classification_time": classification_time,
            "decomposition_time": decomposition_time,
            "retrieval_time": retrieval_time,
            "generation_time": generation_time,
            "total_time": total_time
        }
    }


In [139]:
import pandas as pd
from tqdm import tqdm
import time

# Load your source CSV
df = pd.read_csv("C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\Dataset\\Hotpotqa\\1000_paras_100_queries\\translated_dataset_100_qna.csv")  # Replace with the actual path

# Prepare result list
results = []

# Loop over each question in the DataFrame
for _, row in tqdm(df.iterrows(), total=len(df)):
    query = row["translated_question"]
    answer = row["translated_answer"]
    question_type = row["level"]  # Get 'level' column for the type ('easy' or 'hard')

    try:
        # Run the multihop_handling_LQR function
        result = multihop_handling_LQR(query, question_type)

        # Store the results
        results.append({
            "translated_question": query,
            "translated_answer": answer,
            "classification": result["classification"],
            "retrieved_context": result["retrieved_context"],
            "final_answer": result["final_answer"],
            "classification_time": result["timings"]["classification_time"],
            "decomposition_time": result["timings"]["decomposition_time"],
            "retrieval_time": result["timings"]["retrieval_time"],
            "generation_time": result["timings"]["generation_time"],
            "total_time": result["timings"]["total_time"],
            "level": question_type  # Include the 'level' type
        })

    except Exception as e:
        print(f"Error processing query {query}: {e}")
        # If an error occurs, store the error message in the results
        results.append({
            "translated_question": query,
            "translated_answer": answer,
            "classification": "Error",
            "retrieved_context": "Error",
            "final_answer": "Error",
            "classification_time": 0,
            "decomposition_time": 0,
            "retrieval_time": 0,
            "generation_time": 0,
            "total_time": 0,
            "level": question_type
        })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Save the resulting DataFrame to a CSV
results_df.to_csv("C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\results\\Simple RAG QNA results\\LQR_with_Alif\\LQR_processed_results_with_Alif.csv", index=False,encoding="utf-8-sig")

# Display the resulting DataFrame
print(results_df.head())


  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [1:38:38<00:00, 60.39s/it] 

                                 translated_question translated_answer  \
0  سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بن...         1929ء میں   
1  "آکسفورڈ کالج کے ایک ساتھی ایلک نیلر ڈکن نے کہ...       کرپٹوولوجسٹ   
2  "جراسک پارک کے اداکار ڈیوڈ ہنری ہوانگ نے ""دی ...        بی ڈی وانگ   
3  کون سا کردار ، ڈین کاسٹیلینیٹا کی آواز ، سمپسن...        دادا سمپسن   
4     کون تھا ایک حصہ S#arp، لی Ji-hye یا کرٹس رائٹ؟          لی جی ہے   

  classification                                  retrieved_context  \
0           easy  [سیری بی (اسپانسرشپ وجوہات کی بناء پر سیری بی ...   
1           easy  [۔ ریچر اس وقت اسٹینفورڈ یونیورسٹی میں اسٹینفو...   
2           easy  [۔ یہ فلم والٹر ونجر نے پروڈیوس کی ہے۔ اس فلم ...   
3           easy  [۔ اس کے کریڈٹ میں ""نائٹ آف دی لونگ ڈیڈ"" ، "...   
4           easy  [۔ اس میں ممبر کیم لیپ کا تعارف کیا گیا ہے اور...   

                                        final_answer  classification_time  \
0  جی ہاں، سیاق و سباق درست ہے. سیریز بی (سپانسرش..